<a href="https://colab.research.google.com/github/dbala31/NLP/blob/main/RobertA_metaphor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.37.2
#!pip install transformers[torch]
#!pip install accelerate -U

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from keras import callbacks
from tensorflow.keras import backend
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModelForTokenClassification
from sklearn.model_selection import train_test_split
import pickle
import re

In [ ]:
from transformers import create_optimizer
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Runing on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
test_data=pd.read_csv("/content/drive/MyDrive/VUA_test_all.csv")
train_data=pd.read_csv("/content/drive/MyDrive/VUA_train.csv")

In [ ]:
def format_for_TokenClf(df):
  data_list = []
  sentence=[]
  labels=[]
  for index, row in df.iterrows():
    if row["id"][-2:]=="_1" and index!=0:
      data_list.append((sentence, labels))
      sentence=[]
      labels=[]
    if row["label"]==1:
      label="m"
    else:
      label="l"
    sentence.append(str(row["word"]))
    labels.append(label)
    if index==len(df)-1:
      data_list.append((sentence, labels))

  return data_list

In [ ]:
test_data=format_for_TokenClf(test_data)
train_data=format_for_TokenClf(train_data)

In [ ]:
train_data, val_data= train_test_split(train_data, shuffle=True, test_size=0.05)

In [ ]:
#train
train_tags=[tup[1] for tup in train_data]
train_texts=[tup[0] for tup in train_data]

#val
val_tags=[tup[1] for tup in val_data]
val_texts=[tup[0] for tup in val_data]

#test
test_tags=[tup[1] for tup in test_data]
test_texts=[tup[0] for tup in test_data]

In [ ]:
label_list=["l", "m"]
label_to_id = {l: i for i, l in enumerate(label_list)}
num_labels=len(label_list)
print(num_labels)

2


In [ ]:
model = "roberta-base"

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained("google/electra-large-discriminator")
tokenizer = AutoTokenizer.from_pretrained("roberta-large",add_prefix_space=True, use_fast=True)
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def tokenize_and_align_labels(texts, tags):
  tokenized_inputs = tokenizer(
      texts,
      padding=True,
      truncation=True,
      # We use this argument because the texts in our dataset are lists of words (with a label for each word).
      is_split_into_words=True,
  )
  labels = []
  for i, label in enumerate(tags):
      word_ids = tokenized_inputs.word_ids(batch_index=i)
      previous_word_idx = None
      label_ids = []
      for word_idx in word_ids:
          # Special tokens have a word id that is None. We set the label to -100 so they are automatically
          # ignored in the loss function.
          if word_idx is None:
              label_ids.append(-100)
          # We set the label for the first token of each word.
          elif word_idx != previous_word_idx:
              label_ids.append(label_to_id[label[word_idx]])
          # For the other tokens in a word, we set the label to either the current label or -100, depending on
          # the label_all_tokens flag.
          else:
              label_ids.append(-100)
          previous_word_idx = word_idx

      labels.append(label_ids)
  tokenized_inputs["labels"] = labels
  return tokenized_inputs

In [ ]:
test_input_and_labels = tokenize_and_align_labels(test_texts, test_tags)

val_input_and_labels = tokenize_and_align_labels(val_texts, val_tags)

train_input_and_labels = tokenize_and_align_labels(train_texts, train_tags)

In [ ]:
train_input_data = {
    'input_ids': train_input_and_labels['input_ids'],
    'attention_mask': train_input_and_labels['attention_mask']
    #'special_tokens_mask': train_input_and_labels['special_tokens_mask']
}
valid_input_data = {
    'input_ids': val_input_and_labels['input_ids'],
    'attention_mask': val_input_and_labels['attention_mask']
    #'special_tokens_mask': val_input_and_labels['special_tokens_mask']
}
test_input_data = {
    'input_ids': test_input_and_labels['input_ids'],
    'attention_mask': test_input_and_labels['attention_mask']
    #'special_tokens_mask': test_input_and_labels['special_tokens_mask']
}
y_train = train_input_and_labels["labels"]
y_valid = val_input_and_labels["labels"]
y_test = test_input_and_labels["labels"]

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': train_input_and_labels['input_ids'],
        'attention_mask': train_input_and_labels['attention_mask']
        #'token_type_ids': train_input_and_labels['token_type_ids']
    },
    train_input_and_labels['labels']
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': val_input_and_labels['input_ids'],
        'attention_mask': val_input_and_labels['attention_mask']
        #'token_type_ids': val_input_and_labels['token_type_ids']
    },
    val_input_and_labels['labels']
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': test_input_and_labels['input_ids'],
        'attention_mask': test_input_and_labels['attention_mask']
        #'token_type_ids': test_input_and_labels['token_type_ids']
    },
    test_input_and_labels['labels']
))

NameError: name 'train_input_and_labels' is not defined

In [ ]:
batch_size=32
train_dataset = train_dataset.shuffle(len(train_dataset)).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
model = transformers.TFAutoModelForTokenClassification.from_pretrained("roberta-large",num_labels=num_labels)
optimizers = tf.keras.optimizers.Adam(learning_rate=0.00001)
#loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
#model.compile(optimizer=optimizers, metrics=['accuracy'])
#model.compile(optimizer=optimizers, metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.F1Score(), 'accuracy'])
model.compile(optimizer=optimizers, metrics=['accuracy'])

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForTokenClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tf.config.run_functions_eagerly(True)
#es_cb = callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=5)
n_steps = 20
model.fit(train_dataset, #validation_data=val_dataset,
          batch_size=batch_size,
          steps_per_epoch=n_steps,
          #callbacks=[es_cb],
          epochs=20)
model.save_pretrained(('/content/model_roberta_metaphor_base'))
model.save_pretrained(("/content/drive/MyDrive/Metaphor_roberta_base"))

In [ ]:
## Inference #

In [ ]:

label_list= ['literal',"metaphoric"]
label_dict_relations={ i : l for i, l in enumerate(label_list) }
num_labels = 2
PATH = "/content/drive/MyDrive/Metaphor_roberta_base"
model_metaphor = transformers.TFAutoModelForTokenClassification.from_pretrained(PATH, num_labels=num_labels)

with open(PATH+'/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

All model checkpoint layers were used when initializing TFRobertaForTokenClassification.

All the layers of TFRobertaForTokenClassification were initialized from the model checkpoint at /content/drive/MyDrive/Metaphor_roberta_large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForTokenClassification for predictions without further training.


In [ ]:
MAX_LEN = 256
cust_input = "we have to crack down on what's going on as it is ruining a lot of people"
#cust_input = str1
#cust_input = newdf["Comment Body"][1]
cust_input = f"""  {cust_input} """
cust_input_encodings = tokenizer([cust_input], truncation=True, padding=True,max_length=MAX_LEN,return_tensors='tf')
#cust_input_encodings
cust_input_data = {
    'input_ids': cust_input_encodings['input_ids'],
    'attention_mask': cust_input_encodings['attention_mask'],
    #'token_type_ids': cust_input_encodings['token_type_ids']
  }

In [ ]:
test_predictions = model_metaphor.predict(cust_input_data)
probabilities = tf.nn.softmax(test_predictions.logits, axis=-1)
predicted_labels = tf.argmax(probabilities, axis=-1)
x = tokenizer.convert_ids_to_tokens(cust_input_encodings['input_ids'][0])
predicted_labels.numpy()[0]
predicted_labels_condensed = np.array(predicted_labels.numpy()[0][2:len(predicted_labels.numpy()[0])-2])
words_in_sentence = x[2:len(x)-2]

print(predicted_labels_condensed)
for i in range(0,len(words_in_sentence)):
    if words_in_sentence[i][0] == 'Ġ':
      words_in_sentence[i] = re.sub(r'Ġ', '', words_in_sentence[i])
#words_in_sentence
print(words_in_sentence)



1/1 [==============================] - 14s 14s/step
[0 0 0 1 1 1 0 0 1 1 0 0 0 1 0 0 0 0]
['we', 'have', 'to', 'crack', 'down', 'on', 'what', "'s", 'going', 'on', 'as', 'it', 'is', 'ruining', 'a', 'lot', 'of', 'people']


In [ ]:
print(len(words_in_sentence))
print(sum(predicted_labels_condensed))


18
6


In [ ]:
comments_w_algo=pd.read_csv("/content/drive/MyDrive/comments_w_algorithm.csv")



In [ ]:
newdf = comments_w_algo.head(10)

In [ ]:
newdf["Comment Body"][0]

In [ ]:
len(newdf["Comment Body"])

10

In [ ]:
MAX_LEN = 512
def get_metaphors(cust_input):
  cust_input = f"""  {cust_input} """
  cust_input_encodings = tokenizer([cust_input], truncation=True, padding=True,max_length=MAX_LEN,return_tensors='tf')
  #cust_input_encodings
  cust_input_data = {
      'input_ids': cust_input_encodings['input_ids'],
      'attention_mask': cust_input_encodings['attention_mask'],
      #'token_type_ids': cust_input_encodings['token_type_ids']
      }
  test_predictions = model_metaphor.predict(cust_input_data)
  probabilities = tf.nn.softmax(test_predictions.logits, axis=-1)
  predicted_labels = tf.argmax(probabilities, axis=-1)
  x = tokenizer.convert_ids_to_tokens(cust_input_encodings['input_ids'][0])
  predicted_labels.numpy()[0]
  predicted_labels_condensed = np.array(predicted_labels.numpy()[0][2:len(predicted_labels.numpy()[0])-2])
  words_in_sentence = x[2:len(x)-2]

  #print(predicted_labels_condensed)
  for i in range(0,len(words_in_sentence)):
      if words_in_sentence[i][0] == 'Ġ':
        words_in_sentence[i] = re.sub(r'Ġ', '', words_in_sentence[i])
  #words_in_sentence
  #print(words_in_sentence)

  return(predicted_labels_condensed,words_in_sentence)




In [ ]:
a = []
for i in range(MAX_LEN):
    a.append(i)

a

In [ ]:
#### /content/drive/MyDrive/comments_w_algorithm.csv Analysis ###
prediction_mat_super = []
words_mat_super = []
for j in range(MAX_LEN):
    prediction_mat_super.append(-100)
    words_mat_super.append(-100)

for i in range(0,len(comments_w_algo["Comment Body"])):
  prediction, words = get_metaphors(comments_w_algo["Comment Body"][i])
  prediction_mat = []
  words_mat = []

  for z in range(MAX_LEN):
    prediction_mat.append(-100)
    words_mat.append(-100)

  for k in range(0,len(prediction)):
    prediction_mat[k]=prediction[k]
    words_mat[k] = words[k]

  prediction_mat_super = np.vstack((prediction_mat_super,prediction_mat))
  words_mat_super = np.vstack((words_mat_super,words_mat))



  #print(len(prediction))
  #print(len(words))
  #print("ha")
  #print(sum(prediction))
  print(i)

#print(prediction_mat_super)
#print(words_mat_super)
words_mat_super = np.delete(words_mat_super, (0), axis = 0)
prediction_mat_super = np.delete(prediction_mat_super, (0), axis = 0)
df_word = pd.DataFrame(words_mat_super)
df_prediction = pd.DataFrame(prediction_mat_super)

In [ ]:
pred_csv = df_prediction.to_csv("/content/drive/MyDrive/comments_pred.csv",header=False, index=False)
words_csv = df_word.to_csv("/content/drive/MyDrive/comments_words.csv",header=False, index=False)

In [ ]:
df_word

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,we,have,to,crack,down,on,what,'s,going,on,...,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
1,this,is,also,a,thing,alot,of,people,forget,about,...,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100


In [ ]:
### for Testing  for scores ###

In [ ]:
MAX_LEN = 512
pred_label_array = [0]
GT_array = [0]

for j in range(0,4080):
  #print(j)
  str1 = ""
  text_infer = test_texts[j]
  for i in range(0,len(text_infer)):
    str1 = str1+ " " +text_infer[i]
  #print(str)

  cust_input = str1
  cust_input = f"""  {cust_input} """
  cust_input_encodings = tokenizer([cust_input], truncation=True, padding=True,max_length=MAX_LEN,return_tensors='tf')
  #cust_input_encodings
  cust_input_data = {
      'input_ids': cust_input_encodings['input_ids'],
      'attention_mask': cust_input_encodings['attention_mask'],
      #'token_type_ids': cust_input_encodings['token_type_ids']
    }
  test_predictions = model_metaphor.predict(cust_input_data)
  probabilities = tf.nn.softmax(test_predictions.logits, axis=-1)
  predicted_labels = tf.argmax(probabilities, axis=-1)
  x = tokenizer.convert_ids_to_tokens(cust_input_encodings['input_ids'][0])
  #predicted_labels.numpy()[0]
  #predicted_labels_condensed = np.array(predicted_labels.numpy()[0][1:len(predicted_labels.numpy()[0])-1])
  words_in_sentence = x[3:len(x)-2]

  predicted_labels_condensed = np.array(predicted_labels.numpy()[0][3:len(predicted_labels.numpy()[0])-2])
  GT_condensed = y_test[j][1:len(predicted_labels.numpy()[0])-4]



  for k in range(0,len(predicted_labels_condensed)):
    if GT_condensed[k] != -100:
      pred_label_array = np.vstack((pred_label_array,predicted_labels_condensed[k]))
      GT_array = np.vstack((GT_array,GT_condensed[k]))
    else:
      if predicted_labels_condensed[k] == 1:
        print(j)
        #print(words_in_sentence)
        #print(GT_condensed)
        #print(predicted_labels_condensed)


In [ ]:
print(classification_report(GT_array, pred_label_array))#, target_names=target_names))

precision, recall, f1, _ = precision_recall_fscore_support(GT_array, pred_label_array, average="weighted")
print(precision,recall,f1)

In [ ]:
#### For Electra ###
label_list= ['literal',"metaphoric"]
label_dict_relations={ i : l for i, l in enumerate(label_list) }
num_labels = 2
PATH = "/content/drive/MyDrive/intent_model_electrasmall1"
model_metaphor = transformers.TFAutoModelForTokenClassification.from_pretrained(PATH, num_labels=num_labels)

with open(PATH+'/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

Some layers from the model checkpoint at /content/drive/MyDrive/intent_model_electrasmall1 were not used when initializing TFElectraForTokenClassification: ['dropout_37']
- This IS expected if you are initializing TFElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFElectraForTokenClassification were initialized from the model checkpoint at /content/drive/MyDrive/intent_model_electrasmall1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraForTokenClassification for predictions without further training.


In [ ]:
pred_label_array = [0]
GT_array = [0]

for j in range(0,4080):
  #print(j)
  str1 = ""
  text_infer = test_texts[j]
  for i in range(0,len(text_infer)):
    str1 = str1+ " " +text_infer[i]
  #print(str)

  cust_input = str1
  cust_input = f"""  {cust_input} """
  cust_input_encodings = tokenizer([cust_input], truncation=True, padding=True,max_length=MAX_LEN,return_tensors='tf')
  #cust_input_encodings
  cust_input_data = {
      'input_ids': cust_input_encodings['input_ids'],
      'attention_mask': cust_input_encodings['attention_mask'],
      'token_type_ids': cust_input_encodings['token_type_ids']
    }
  test_predictions = model_metaphor.predict(cust_input_data)
  probabilities = tf.nn.softmax(test_predictions.logits, axis=-1)
  predicted_labels = tf.argmax(probabilities, axis=-1)
  x = tokenizer.convert_ids_to_tokens(cust_input_encodings['input_ids'][0])
  #predicted_labels.numpy()[0]
  #predicted_labels_condensed = np.array(predicted_labels.numpy()[0][1:len(predicted_labels.numpy()[0])-1])
  words_in_sentence = x[1:len(x)-1]

  predicted_labels_condensed = np.array(predicted_labels.numpy()[0][1:len(predicted_labels.numpy()[0])-1])
  GT_condensed = y_test[j][1:len(predicted_labels.numpy()[0])-1]



  for k in range(0,len(predicted_labels_condensed)):
    if GT_condensed[k] != -100:
      pred_label_array = np.vstack((pred_label_array,predicted_labels_condensed[k]))
      GT_array = np.vstack((GT_array,GT_condensed[k]))
    else:
      if predicted_labels_condensed[k] == 1:
        print(j)
        #print(words_in_sentence)
        #print(GT_condensed)
        #print(predicted_labels_condensed)


In [ ]:
### For Electra_Base____ ###
print(classification_report(GT_array, pred_label_array))#, target_names=target_names))

precision, recall, f1, _ = precision_recall_fscore_support(GT_array, pred_label_array, average="weighted")
print(precision,recall,f1)

              precision    recall  f1-score   support

           0       0.94      0.99      0.96     51541
           1       0.88      0.49      0.63      6819

    accuracy                           0.93     58360
   macro avg       0.91      0.74      0.80     58360
weighted avg       0.93      0.93      0.92     58360

0.9291011488080951 0.9321967100753941 0.9235382049041498


In [ ]:
MAX_LEN = 512
def get_metaphors_electra(cust_input):
  cust_input = f"""  {cust_input} """
  cust_input_encodings = tokenizer([cust_input], truncation=True, padding=True,max_length=MAX_LEN,return_tensors='tf')
  #cust_input_encodings
  cust_input_data = {
      'input_ids': cust_input_encodings['input_ids'],
      'attention_mask': cust_input_encodings['attention_mask'],
      #'token_type_ids': cust_input_encodings['token_type_ids']
      }
  test_predictions = model_metaphor.predict(cust_input_data)
  probabilities = tf.nn.softmax(test_predictions.logits, axis=-1)
  predicted_labels = tf.argmax(probabilities, axis=-1)
  x = tokenizer.convert_ids_to_tokens(cust_input_encodings['input_ids'][0])
  predicted_labels.numpy()[0]
  predicted_labels_condensed = np.array(predicted_labels.numpy()[0][1:len(predicted_labels.numpy()[0])-1])
  words_in_sentence = x[1:len(x)-1]

  #print(predicted_labels_condensed)
  #for i in range(0,len(words_in_sentence)):
  #    if words_in_sentence[i][0] == 'Ġ':
  #      words_in_sentence[i] = re.sub(r'Ġ', '', words_in_sentence[i])
  #words_in_sentence
  #print(words_in_sentence)

  return(predicted_labels_condensed,words_in_sentence)

In [ ]:
#### /content/drive/MyDrive/comments_w_algorithm.csv Analysis ###
prediction_mat_super = []
words_mat_super = []
for j in range(MAX_LEN):
    prediction_mat_super.append(-100)
    words_mat_super.append(-100)

for i in range(0,len(comments_w_algo["Comment Body"])):
#for i in range(0,1):
  prediction, words = get_metaphors_electra(comments_w_algo["Comment Body"][i])
  prediction_mat = []
  words_mat = []

  for z in range(MAX_LEN):
    prediction_mat.append(-100)
    words_mat.append(-100)

  for k in range(0,len(prediction)):
    prediction_mat[k]=prediction[k]
    words_mat[k] = words[k]

  prediction_mat_super = np.vstack((prediction_mat_super,prediction_mat))
  words_mat_super = np.vstack((words_mat_super,words_mat))



  #print(len(prediction))
  #print(len(words))
  #print("ha")
  #print(sum(prediction))
  print(i)

#print(prediction_mat_super)
#print(words_mat_super)
words_mat_super = np.delete(words_mat_super, (0), axis = 0)
prediction_mat_super = np.delete(prediction_mat_super, (0), axis = 0)
df_word = pd.DataFrame(words_mat_super)
df_prediction = pd.DataFrame(prediction_mat_super)

1506
1/1 [==============================] - 0s 53ms/step
1507
1/1 [==============================] - 0s 54ms/step
1508
1/1 [==============================] - 0s 94ms/step
1509
1/1 [==============================] - 0s 56ms/step
1510
1/1 [==============================] - 0s 48ms/step
1511
1/1 [==============================] - 0s 49ms/step
1512
1/1 [==============================] - 0s 48ms/step
1513
1/1 [==============================] - 0s 55ms/step
1514
1/1 [==============================] - 0s 44ms/step
1515
1/1 [==============================] - 0s 50ms/step
1516
1/1 [==============================] - 0s 50ms/step
1517
1/1 [==============================] - 0s 51ms/step
1518
1/1 [==============================] - 0s 46ms/step
1519
1/1 [==============================] - 0s 46ms/step
1520
1/1 [==============================] - 0s 93ms/step
1521
1/1 [==============================] - 0s 45ms/step
1522
1/1 [==============================] - 0s 61ms/step
1523
1/1 [=====================

In [ ]:
df_word
df_prediction

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0,0,0,1,0,1,0,0,0,0,...,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100


In [ ]:
pred_csv_electra = df_prediction.to_csv("/content/drive/MyDrive/comments_pred_electra.csv",header=False, index=False)
words_csv_electra = df_word.to_csv("/content/drive/MyDrive/comments_words_electra.csv",header=False, index=False)